Reading the verdict file 

In [1]:
with open("the-verdict.txt", 'r', encoding = "utf-8") as f:
    raw_text = f.read()
print("Total number of characters:", len(raw_text))
print(raw_text[:100])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [3]:
import re
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip() != '']
print(len(preprocessed))
print(preprocessed[:100])

4649
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--']


In [4]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1159


In [5]:
vocab = {token:integer for integer, token in enumerate(all_words)}


In [9]:
class SimpleTokenizerV1: 
    def __init__(self, vocab): 
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text): 
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip() != '']
        return [self.str_to_int[token] for token in preprocessed]
    
    def decode(self, tokens): 
        text =  ' '.join([self.int_to_str[token] for token in tokens])
        text = re.sub(r'\s+([,.?_!"()\'])', r'\1', text)    # Removes unnecessary spaces
        return text 

In [10]:
tokenizer = SimpleTokenizerV1(vocab)

text = """It's the last he painted, you know."""
ids = tokenizer.encode(text)
print(ids)

[58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 7]


In [11]:
print(tokenizer.decode(ids))

It' s the last he painted, you know.


Byte Pair Encoding

In [13]:
%pip install tiktoken 

Looking in indexes: https://pypi.org/simple, https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import tiktoken 
from importlib.metadata import version
print(version('tiktoken'))

0.8.0


In [16]:
tokenizer = tiktoken.get_encoding('gpt2')
text = """It's the last he painted, you know."""
ids = tokenizer.encode(text)
print(ids)
strings = tokenizer.decode(ids)
print(strings)


[1026, 338, 262, 938, 339, 13055, 11, 345, 760, 13]
It's the last he painted, you know.


Importing Dataset and Dataloader

In [17]:
import torch 
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset): 
    def __init__(self, text, tokenizer, max_length, stride): 
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(text)
        
        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self): 
        return len(self.input_ids)
    
    def __getitem__(self, idx): 
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(text, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0): 
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    return DataLoader(dataset, batch_size = batch_size, shuffle = shuffle, drop_last = drop_last, num_workers = num_workers)

In [24]:
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 4, shuffle = False )

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [25]:
output_dim = 256 
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [29]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 128, shuffle = True, drop_last = True)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print(inputs.shape)

tensor([[  383,  8631,  3872,   373],
        [   11,   508,   550, 18459],
        [ 2612,  4369,    11,   523],
        [  314,   550,  1775,   683],
        [  198,  1544, 13818,  4622],
        [ 1021,   757,   438, 10919],
        [  262,  1633,   286, 24380],
        [  616,   705, 23873,  2350]])
torch.Size([8, 4])


In [30]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [31]:
context_length = max_length 
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [32]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
